In [1]:
from load_split import load_and_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import models
from keras import layers
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix,classification_report

In [3]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = load_and_split('fer_no_nf.csv')
class_weights = np.load('class_weights.npy')
class_labels = ('neutral','happiness','surprise','sadness','anger','disgust','fear','contempt','unknown')

In [8]:
inputs = layers.Input(shape=(48, 48, 1))

x = layers.Rescaling(1./255)(inputs)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.2)(x)

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Dropout(0.2)(x)

x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.6)(x)

outputs = layers.Dense(9, activation='softmax')(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_3 (Rescaling)         │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 48, 48, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 48, 48, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 48, 48, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 24, 24, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       294,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 434,537 (1.66 MB)

 Trainable params: 434,473 (1.66 MB)

 Non-trainable params: 64 (256.00 B)

In [9]:
history = model.fit(x_train, y_train,                  
                    epochs=10,
                    batch_size = 32,
                    validation_data=(x_val , y_val),
                    class_weight={index:weight for index,weight in enumerate(class_weights)})
model.save('big_weighted.keras')

Epoch 1/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 21s 22ms/step - accuracy: 0.3132 - loss: 14.4349 - val_accuracy: 0.3917 - val_loss: 1.8896
Epoch 2/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.4076 - loss: 13.3481 - val_accuracy: 0.5156 - val_loss: 1.7793
Epoch 3/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - accuracy: 0.4950 - loss: 12.7123 - val_accuracy: 0.6271 - val_loss: 1.5007
Epoch 4/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.5713 - loss: 11.9094 - val_accuracy: 0.6439 - val_loss: 1.4314
Epoch 5/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.5687 - loss: 12.0389 - val_accuracy: 0.6705 - val_loss: 1.3189
Epoch 6/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.6141 - loss: 11.1673 - val_accuracy: 0.6809 - val_loss: 1.3094
Epoch 7/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.6223 - loss: 10.9102 - val_accuracy: 0.6867 - val_loss: 1.3213
Epoch 8/10
890/890 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.6284 - loss: 10